## Project Setup

In [1]:
# !git clone https://github.com/mozartfish/dd2417-dating-historical-texts.git

In [2]:
import os
import random
random.seed(42)
# local path
raw_dataset_path = './Datasets/raw_data'
print(os.listdir(raw_dataset_path))

# google drive path
# raw_dataset_path = './dd2417-dating-historical-texts/Datasets/raw_data/'
# print(os.listdir('./dd2417-dating-historical-texts/'))

['1890', '1830', '1800', '1720', '.DS_Store', '1780', '1710', '1750', '1760', '1840', '1870', '1860', '1850', '1770', '1740', '1700', '1730', '1790', '1810', '1880', '1820']


## Data Preprocessing

In [3]:
# create new directory to store cleaned texts
# directory='./dd2417-dating-historical-texts/Datasets/cleaned_data'
clean_dataset_path = './Datasets/cleaned_data'
random_passage_path = './Datasets/random_passages'
if not os.path.exists(clean_dataset_path):
  os.makedirs('./Datasets/cleaned_data')

if not os.path.exists(random_passage_path):
  os.makedirs('./Datasets/random_passages')

In [4]:
import re 
import string
def clean_text(text):
  # remove everything before start
  start_match = re.search(r'\*\*\* START OF.*?\*\*\*', text, re.IGNORECASE | re.DOTALL)
  if start_match:
    text = text[start_match.end():]

  # remove everything after end
  end_match = re.search(r'\*\*\* END OF.*?\*\*\*', text, re.IGNORECASE | re.DOTALL)
  if end_match:
    text = text[:end_match.start()]

  # remove years 1000's
  text = re.sub(r'\b1[0-9]{3}\b', '', text)

  # remove whitespace
  text = re.sub(r'\s+', ' ', text)

  return text.strip()


In [5]:
years = [1700, 1710, 1720, 1730, 1740, 1750, 1760, 1770, 1780, 1790]

In [6]:
def preprocess_text(dataset_path, year):
  print(f"preprocess text")
  print(f"directory year: {year}")
  decade_path = dataset_path + "/" + str(year) + "/"
  cleaned_data_path = f'./Datasets/cleaned_data/{year}/'
  if not os.path.exists(cleaned_data_path):
    os.makedirs(cleaned_data_path)
  # process and clean each text file
  text_list = os.listdir(decade_path)
  for file in text_list:
    if file.endswith('.txt'):
      print(f"file name: {file}")

      # read file
      with open(decade_path + file, 'r', encoding='utf-8') as f:
        raw_text = f.read()

      cleaned_text = clean_text(raw_text)

      # save file
      out_file = cleaned_data_path + file
      with open(out_file, 'w', encoding='utf-8') as f:
        f.write(cleaned_text)
        print(f"cleaned text and save to -> {out_file}")
        print()

### 1700's Texts

In [7]:
years = [1700, 1710, 1720, 1730, 1740, 1750, 1760, 1770, 1780, 1790]
# test_year = years[0]
# preprocess_text(raw_dataset_path, test_year)
for year in years:
  preprocess_text(raw_dataset_path, year)


preprocess text
directory year: 1700
file name: 1700_5.txt
cleaned text and save to -> ./Datasets/cleaned_data/1700/1700_5.txt

file name: 1700_4.txt
cleaned text and save to -> ./Datasets/cleaned_data/1700/1700_4.txt

file name: 1700_1.txt
cleaned text and save to -> ./Datasets/cleaned_data/1700/1700_1.txt

file name: 1700_3.txt
cleaned text and save to -> ./Datasets/cleaned_data/1700/1700_3.txt

file name: 1700_2.txt
cleaned text and save to -> ./Datasets/cleaned_data/1700/1700_2.txt

preprocess text
directory year: 1710
file name: 1710_4.txt
cleaned text and save to -> ./Datasets/cleaned_data/1710/1710_4.txt

file name: 1710_5.txt
cleaned text and save to -> ./Datasets/cleaned_data/1710/1710_5.txt

file name: 1710_2.txt
cleaned text and save to -> ./Datasets/cleaned_data/1710/1710_2.txt

file name: 1710_3.txt
cleaned text and save to -> ./Datasets/cleaned_data/1710/1710_3.txt

file name: 1710_1.txt
cleaned text and save to -> ./Datasets/cleaned_data/1710/1710_1.txt

preprocess text


### 1800's Texts

In [8]:
years = [1800, 1810, 1820, 1830, 1840, 1850, 1860, 1870, 1880, 1890]
for year in years:
  preprocess_text(raw_dataset_path, year)

preprocess text
directory year: 1800
file name: 1800_2.txt
cleaned text and save to -> ./Datasets/cleaned_data/1800/1800_2.txt

file name: 1800_3.txt
cleaned text and save to -> ./Datasets/cleaned_data/1800/1800_3.txt

file name: 1800_1.txt
cleaned text and save to -> ./Datasets/cleaned_data/1800/1800_1.txt

file name: 1800_4.txt
cleaned text and save to -> ./Datasets/cleaned_data/1800/1800_4.txt

file name: 1800_5.txt
cleaned text and save to -> ./Datasets/cleaned_data/1800/1800_5.txt

file name: 1800_6.txt
cleaned text and save to -> ./Datasets/cleaned_data/1800/1800_6.txt

preprocess text
directory year: 1810
file name: 1810_1.txt
cleaned text and save to -> ./Datasets/cleaned_data/1810/1810_1.txt

file name: 1810_3.txt
cleaned text and save to -> ./Datasets/cleaned_data/1810/1810_3.txt

file name: 1810_2.txt
cleaned text and save to -> ./Datasets/cleaned_data/1810/1810_2.txt

file name: 1810_5.txt
cleaned text and save to -> ./Datasets/cleaned_data/1810/1810_5.txt

file name: 1810_

### Generate Random Text Snippets


In [ ]:

def generate_random_passage(text, passage_length=150):
    words = text.split()
    start = random.randint(0, len(words) - passage_length)
    random_passage_words = words[start : start + passage_length]
    random_passage = ' '. join(random_passage_words)
    return random_passage 

In [ ]:
def create_dataset(dataset_path, year, num_passages=4, passage_length=1200):
    print(f"create dataset")
    print(f"directory year: {year}")
    decade_path = dataset_path + "/" + str(year) + "/"
    random_passage_path = f"./Datasets/random_passages/{year}/"
    if not os.path.exists(random_passage_path):
        os.makedirs(random_passage_path)
    # process and clean each text file
    text_list = os.listdir(decade_path)
    for file in text_list:
        if file.endswith(".txt"):
            print(f"file name: {file}")
            # read file
            with open(decade_path + file, "r", encoding="utf-8") as f:
                clean_text = f.read()

                for i in range(num_passages):
                    passage = generate_random_passage(clean_text, passage_length)
                    out_file = (
                        random_passage_path + file.strip(".txt") + str(i) + ".txt"
                    )
                    with open(out_file, "w", encoding="utf-8") as f:
                        f.write(passage)
                        print(f"random passage and save to -> {out_file}")
                        print()

In [14]:
years = [1700, 1710, 1720, 1730, 1740, 1750, 1760, 1770, 1780, 1790]
for year in years:
    create_dataset(clean_dataset_path, year)

create dataset
directory year: 1700
file name: 1700_5.txt
random passage and save to -> ./Datasets/random_passages/1700/1700_50.txt

random passage and save to -> ./Datasets/random_passages/1700/1700_51.txt

random passage and save to -> ./Datasets/random_passages/1700/1700_52.txt

random passage and save to -> ./Datasets/random_passages/1700/1700_53.txt

file name: 1700_4.txt
random passage and save to -> ./Datasets/random_passages/1700/1700_40.txt

random passage and save to -> ./Datasets/random_passages/1700/1700_41.txt

random passage and save to -> ./Datasets/random_passages/1700/1700_42.txt

random passage and save to -> ./Datasets/random_passages/1700/1700_43.txt

file name: 1700_1.txt
random passage and save to -> ./Datasets/random_passages/1700/1700_10.txt

random passage and save to -> ./Datasets/random_passages/1700/1700_11.txt

random passage and save to -> ./Datasets/random_passages/1700/1700_12.txt

random passage and save to -> ./Datasets/random_passages/1700/1700_13.txt

In [15]:
years = [1800, 1810, 1820, 1830, 1840, 1850, 1860, 1870, 1880, 1890]
for year in years:
    create_dataset(clean_dataset_path, year)

create dataset
directory year: 1800
file name: 1800_2.txt
random passage and save to -> ./Datasets/random_passages/1800/1800_20.txt

random passage and save to -> ./Datasets/random_passages/1800/1800_21.txt

random passage and save to -> ./Datasets/random_passages/1800/1800_22.txt

random passage and save to -> ./Datasets/random_passages/1800/1800_23.txt

file name: 1800_3.txt
random passage and save to -> ./Datasets/random_passages/1800/1800_30.txt

random passage and save to -> ./Datasets/random_passages/1800/1800_31.txt

random passage and save to -> ./Datasets/random_passages/1800/1800_32.txt

random passage and save to -> ./Datasets/random_passages/1800/1800_33.txt

file name: 1800_1.txt
random passage and save to -> ./Datasets/random_passages/1800/1800_10.txt

random passage and save to -> ./Datasets/random_passages/1800/1800_11.txt

random passage and save to -> ./Datasets/random_passages/1800/1800_12.txt

random passage and save to -> ./Datasets/random_passages/1800/1800_13.txt